In [1]:
from astroquery.skyview import SkyView
from astroquery.vizier import Vizier
from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import cv2
import numpy as np
from astropy.io import fits
import os
import pandas as pd

In [2]:
# Define the coordinates for the Eagle Nebula Pillars of Creation
coords = SkyCoord('18h18m48s -13d49m00s', frame='icrs')
# Track the name of the object
object_name = 'Eagle Nebula Pillars of Creation'

In [71]:
# Fetch an image from SkyView
image_list = SkyView.get_images(position=coords, survey=['DSS'], radius=0.1 * u.deg, pixels=[6000, 6000])
image_hdu = image_list[0][0]
image = image_list[0][0].data

In [55]:
# Create a folder for the object with the name of the object with underscores instead of spaces
object_name = object_name.replace(' ', '_')
os.makedirs(f'images/{object_name}', exist_ok=True)

In [56]:
# Save the image as a FITS file
fits_file_path = f'images/{object_name}/{object_name}.fits'
hdu = fits.PrimaryHDU(image)
hdul = fits.HDUList([hdu])
hdul.writeto(fits_file_path, overwrite=True)

In [57]:
# # Display the image
# plt.imshow(image, cmap='gray')

In [63]:
# Download the image in all available color maps to the object folder
for cmap in plt.colormaps():
    plt.imsave(f'images/{object_name}/{object_name}_{cmap}.png', image, cmap=cmap)

In [34]:
# # Visualize al color maps in a single image
# fig, ax = plt.subplots(1, 1, figsize=(12, 12))
# for i, cmap in enumerate(plt.colormaps()):
#     ax.imshow(image, cmap=cmap)
#     ax.set_title(cmap)
#     plt.axis('off')
#     plt.savefig(f'images/{object_name}/All_cmaps__{object_name}.png')

In [ ]:
image_hdu.header

In [ ]:
# Extract WCS information
wcs = WCS(image_hdu.header)

# Display the image with WCS projection and grid
fig = plt.figure(figsize=(14, 14), dpi=600)
ax = fig.add_subplot(111, projection=wcs)
ax.imshow(image_hdu.data, cmap='gray', origin='lower')
ax.set_title(f'{object_name}')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
ax.grid(color='white', ls='dotted')


# # Display the image without WCS projection
# plt.imshow(image, cmap='gray')
# plt.title('Sky Image')
# plt.xlabel('RA')
# plt.ylabel('Dec')
# plt.show()

In [ ]:
# Display the image with WCS projection and grid
fig = plt.figure(figsize=(14, 14), dpi=600)
ax = fig.add_subplot(111, projection=wcs)
ax.imshow(image_hdu.data, cmap='twilight_shifted_r', origin='lower')
ax.set_title(f'{object_name}')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
ax.grid(color='white', ls='dotted')

In [ ]:
# Fetch star data from Vizier using the 2MASS catalog
v = Vizier(columns=['*'])
catalog_list = v.query_region(coords, radius=0.4 * u.deg, catalog='II/246')
catalog = catalog_list[0]

# Convert the table to a pandas DataFrame for easier manipulation
catalog = catalog.to_pandas()
print(catalog)

# Save the catalog as a CSV file
catalog.to_csv(f'images/{object_name}/2MASS_catalog_{object_name}.csv', index=False)

In [ ]:
# Fetch star data from Vizier using the Hipparcos catalog
v = Vizier(columns=['*'])
catalog_list = v.query_region(coords, radius=2 * u.deg, catalog='I/239/hip_main')
catalog = catalog_list[0]

# Covnert the table to a pandas DataFrame for easier manipulation
catalog = catalog.to_pandas()
print(catalog)

# Save the catalog as a CSV file
catalog.to_csv(f'images/{object_name}/Hipparcos_catalog_{object_name}.csv', index=False)

In [13]:
# Example preprocessing steps
# Normalize the image
normalized_image = (image - np.min(image)) / (np.max(image) - np.min(image))

In [ ]:
# Extract star positions and magnitudes
star_positions = np.array([stars['RAJ2000'], stars['DEJ2000']]).T
star_magnitudes = stars['Vmag']

In [ ]:
# Save preprocessed data for further analysis
np.save('preprocessed_image.npy', normalized_image)
star_data = pd.DataFrame({'RA': stars['RAJ2000'], 'Dec': stars['DEJ2000'], 'Magnitude': stars['Vmag']})
star_data.to_csv('preprocessed_star_data.csv', index=False)

In [ ]:
# Save the image using OpenCV
cv2.imwrite('sky_image.png', image)